In [7]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
from torch import embedding
from nltk.corpus import stopwords
from nltk.corpus import wordnet 
import nltk
import numpy as np
from sklearn import preprocessing
import math
import random
from random import shuffle
from sklearn import preprocessing


In [8]:
class Subject():
    def __init__(self,n):
        self.subject_number = n
        global df
        df = pd.read_csv("data/Text/{0:0>3n}(done).csv".format(n))
        self.words = df["Words"]
        self.start_time = df["Start_time"]
        self.shuffled_words_list = []  #已經經過shuffled的word list
        for i in range(len(self.words)):
            self.shuffled_words_list.append(self.words[i])
        shuffle(self.shuffled_words_list)
        for i in range(len(self.shuffled_words_list)):
            df["Words"][i] = self.shuffled_words_list[i]
        df.to_csv(f'data/Text/{n:0>3d}(done_shuffled).csv')
    ##########################################################################################
    def shuffled_calculate(self,destination,timeframe,space,start_time=0,end=480):
        global df
        df['Start_time'] = pd.to_numeric(df['Start_time'])
        #df['end_time'] = pd.to_numeric(df['end_time'])
        global all_sentences001
        all_sentences001 = []
        t=start_time
        
        while t<end:
            sentences = []
            for index, rows in df.loc[(df["Start_time"]>=t) & (df["Start_time"]<=(t+timeframe))].iterrows():
                sentences.append(rows["Words"])
            all_sentences001.append(" ".join(sentences))
            t+=space

        
        model = SentenceTransformer('all-MiniLM-L6-v2')
        global SBERT 
        SBERT= []

        def get_cosine_similarity(feature_vec_1, feature_vec_2):    
            return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]
        # Two lists of sentences
        for i in range(0,len(all_sentences001)-1):
            #Compute embedding for both lists
            embeddings1 = model.encode(all_sentences001[i], convert_to_tensor=True)
            #print(all_sentences001[i])
            embeddings2 = model.encode(all_sentences001[i+1], convert_to_tensor=True)
            #print(all_sentences001[i+1])
            #Compute cosine-similarities
            cosine_scores = util.cos_sim(embeddings1, embeddings2)
            #print(cosine_scores)
            # convert from PyTorch tensor to numpy array
            SBERT.append(get_cosine_similarity(embeddings1, embeddings2))
        ######
            print("timeframe: ",i,":",i+10,"\n")
            print(get_cosine_similarity(embeddings1, embeddings2),"\n")
            print(all_sentences001[i],"\n")
            print(all_sentences001[i+1],"\n")
            print()

        
        all_sentences001 = []
        stops = set(stopwords.words('english'))
        t=start_time
        while t <end:
            sentences = []
            for index, rows in df.loc[(df["Start_time"]>=t) & (df["Start_time"]<=(t+timeframe))].iterrows():
                if rows["Words"].lower() not in stops:
                    sentences.append(rows["Words"].lower())
                    #print(rows["Words"])
            all_sentences001.append(" ".join(sentences))
            #print(t)
            t += space 
        ''' 舊的sample step
        for t in range(0,480,0.5):
            sentences = []
            for index, rows in df.loc[(df["Start_time"]>=t) & (df["Start_time"]<=(t+10))].iterrows():
                if rows["Words"].lower() not in stops:
                    sentences.append(rows["Words"].lower())
            all_sentences001.append(" ".join(sentences))
        '''    
        #nltk.download('omw-1.4')
        #nltk.download('wordnet')
        #nltk.download('stopwords')
        global pair_counts
        pair_counts = []
        stops = set(stopwords.words('english'))
        global all_sentences_wu_sum
        all_sentences_wu_sum = []
        global all_sentences_wu_sum_divided
        all_sentences_wu_sum_divided = []
        for sentence_num in range(0,len(all_sentences001)): #現在第幾句
            wu_pulmer_sum = 0
            the_sentence = str(all_sentences001[sentence_num]).split()
            #print(the_sentence)
            timeframe_pairs_count = 0
            for words_num in range(0, (len(the_sentence)-1)): #標定第一個字
                if len(wordnet.synsets(the_sentence[words_num])) > 0 :
                    syns1 = wordnet.synsets(the_sentence[words_num])
                    syns1_num = len(syns1)
                else:
                    continue
                for words_num2 in range((words_num+1), len(the_sentence)): #標定第二個字
                    if len(wordnet.synsets(the_sentence[words_num2])) > 0 :
                        syns2 = wordnet.synsets(the_sentence[words_num2])
                        syns2_num = len(syns2)
                        for i in range(0, syns1_num):
                            for u in range(0, syns2_num):
                                #print(the_sentence[words_num],"的",syns1[i]," ", the_sentence[words_num2],"的",syns2[u]," " ,syns1[i].wup_similarity(syns2[u]))
                                wu_pulmer_sum += syns1[i].wup_similarity(syns2[u])
                                timeframe_pairs_count += 1
                    else:
                        continue
            if timeframe_pairs_count != 0:
                all_sentences_wu_sum_divided.append(wu_pulmer_sum/timeframe_pairs_count) #算出除以pairs數量的平均值
            else: 
                all_sentences_wu_sum_divided.append(wu_pulmer_sum)
            all_sentences_wu_sum.append(wu_pulmer_sum)
            pair_counts.append(timeframe_pairs_count)
            #print(all_sentences_wu_sum)
            global wu_difference
            wu_difference = []
            for x in range(len(all_sentences_wu_sum)-1):
                wu_difference.append(all_sentences_wu_sum[x+1]-all_sentences_wu_sum[x])
        results_csv = pd.DataFrame(list(zip(SBERT,all_sentences_wu_sum,all_sentences_wu_sum_divided,pair_counts)), columns = ['SBERT',"Wu-Pulmer","Wu-Pulmer_divided","pair_counts"])
        results_csv.to_csv("data/Text/{0:0>3d}({1}).csv".format(self.subject_number,destination))
        


注意出來的self.words已經是自動打亂的結果了

In [9]:
subject8 = Subject(8)

C:\Users\chang\AppData\Local\Temp\ipykernel_6240\3882780971.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Words"][i] = self.shuffled_words_list[i]


In [10]:
print(subject8.shuffled_words_list)

['standing', 'recognized', 'knew', 'that', 'about', 'what', 'am', 'between', 'obviously', 'patch', 'I', 'makes', 'start', 'for', 'and', 'and', 'everything', "don't", 'of', 'going', 'and', 'say', 'be', 'worried', 'that', 'I', 'personal', 'happy', 'would', 'being', 'seems', 'day.', 'person', 'of', 'so', 'an', 'first', 'Triads', 'rough', 'just', 'some', 'jerk', 'by', 'I', 'what', 'of', 'always', 'new', 'doing', 'like', 'I', 'choice', 'talk', 'here', 'being', 'between', 'ago', 'do', 'their', 'stupid', 'was', 'friends', 'anything', 'it', 'to', 'have', 'my', 'the', 'that', 'guy', "I've", 'classes', 'kind', 'making', 'just', 'it', 'know', 'started', 'simple', 'I', 'proposed', 'plan', 'I', 'other', 'all', 'who', 'part', 'people', 'focusing', "here's", 'appreciate', 'well', 'to', 'is', 'of', 'most', "I've", "you're", 'go', 'And', "I'm", 'some', 'wanted', 'just', 'guess', 'of', 'with', 'my', 'a', 'and', 'to', 'a', 'a', 'people', 'time', 'on', 'not', 'kinds', 'I', 'routine', 'passion', 'why', 'pr

def shuffled_calculate(self,destination,timeframe,space,start_time=0,end=480):

In [11]:
#subject8.shuffled_calculate("shuffled",10,1)

In [12]:
subject8 = Subject(10)
subject8.shuffled_calculate("shuffled",10,1)

C:\Users\chang\AppData\Local\Temp\ipykernel_6240\3882780971.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Words"][i] = self.shuffled_words_list[i]


timeframe:  0 : 10 

1.0 

as thing when beautiful however else a feel of urban gym and in which and it 

as thing when beautiful however else a feel of urban gym and in which and it 


timeframe:  1 : 11 

0.9959557 

as thing when beautiful however else a feel of urban gym and in which and it 

as thing when beautiful however else a feel of urban gym and in which and it but and 


timeframe:  2 : 12 

0.94945633 

as thing when beautiful however else a feel of urban gym and in which and it but and 

thing when beautiful however else a feel of urban gym and in which and it but and need 


timeframe:  3 : 13 

0.93848103 

thing when beautiful however else a feel of urban gym and in which and it but and need 

thing when beautiful however else a feel of urban gym and in which and it but and need world ways enjoy 


timeframe:  4 : 14 

0.8961387 

thing when beautiful however else a feel of urban gym and in which and it but and need world ways enjoy 

a feel of urban gym and in which a

KeyboardInterrupt: 